<a href="https://colab.research.google.com/github/jloutz/Resume_NER/blob/master/flair_nlp_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Resume NER Part 4: Working with Flair NLP

---

In this part we will use flair NLP to train a model on our data and evaluate the results. Please make sure you have set up your Google account and uploaded your files to Google drive. This Notebook should run on Google Colab.

Let's change the working directory to the Google drive where our training data is, and install flair nlp. 

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
import os
os.chdir("/content/gdrive/My Drive/SAKI_2019/dataset") 

In [0]:
# download flair library #
! pip install flair

In the next section, we will train a NER model with flair. This code is taken from the flair nlp tutorials section 7. "Training a model" 
https://github.com/zalandoresearch/flair/blob/master/resources/docs/TUTORIAL_7_TRAINING_A_MODEL.md



In [0]:
# imports 
from flair.datasets import Corpus
from flair.data_fetcher import NLPTaskDataFetcher

## make sure this describes your file structure
columns = {0: 'text', 1: 'ner'}

# folder where training and test data are
data_folder = '/content/gdrive/My Drive/SAKI_2019/dataset/flair'

# 1.0 is full data, try a much smaller number like 0.1 to test run the code
downsample = 0.1 

## your train file name
train_file = 'train_res_bilou_2.txt'

## your test file name
test_file = 'test_res_bilou_2.txt'
# 1. get the corpus
corpus: Corpus = NLPTaskDataFetcher.load_column_corpus(data_folder, columns,
                                                             train_file=train_file,
                                                             test_file=test_file,
                                                           dev_file=None).downsample(downsample)
print(corpus)

# 3. make the tag dictionary from the corpus
tag_dictionary = corpus.make_tag_dictionary(tag_type='ner')
print(tag_dictionary.idx2item)


In [0]:

# 4. initialize embeddings. Experiment with different embedding types to see what gets the best results
from flair.embeddings import TokenEmbeddings, WordEmbeddings, StackedEmbeddings,FlairEmbeddings

embedding_types: List[TokenEmbeddings] = [
    WordEmbeddings('glove'),
    # comment in this line to use character embeddings
    # CharacterEmbeddings(),

    # comment in these lines to use flair embeddings (needs a LONG time to train :-)
    #FlairEmbeddings('news-forward'),
    #FlairEmbeddings('news-backward'),
]

embeddings: StackedEmbeddings = StackedEmbeddings(embeddings=embedding_types)

# 5. initialize sequence tagger
from flair.models import SequenceTagger

tagger: SequenceTagger = SequenceTagger(hidden_size=256,
                                        embeddings=embeddings,
                                        tag_dictionary=tag_dictionary,
                                        tag_type='ner',
                                        use_crf=True)

In [0]:
# 6. initialize trainer
from flair.trainers import ModelTrainer

trainer: ModelTrainer = ModelTrainer(tagger, corpus)

## give your model a name and folder of your choice. Your model will be saved there for loading later 
## you can run this notebook many times with different embeddings/params and save the models with different names
model_name = 'resources/taggers/resume-ner-4'

# 7. start training - you can experiment with batch size if you get memory errors
# how many epochs does it take before the model stops showing improvement? Start with a big number like 150, and stop the code cell
# from running at any time - the framework will persist the best model even if you interrupt training. 
trainer.train(model_name,
              learning_rate=0.1,
              mini_batch_size=32,
              #anneal_with_restarts=True,
              max_epochs=150)


